In [ ]:
%run import_libs.py

### get data

In [ ]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')
num_features = pd.read_csv("num_feats_after_filtering.csv")["0"].to_list()

df_train_agg = get_df_w_aggrs(df=df_train, feats=num_features)
df_train_target = get_target(TARGET_PATH='./data/train_labels.csv')
df_train = get_train_data_with_target_merged(df_train=df_train_agg, df_train_target=df_train_target)

In [ ]:
df_test = get_test_data(TEST_PATH='./data/test.parquet')
df_test = get_df_w_aggrs(df=df_test, feats=num_features)

In [ ]:
cat_features = [f"{f}_last" for f in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
cat_features

In [ ]:
payment_feats = []
delinq_feats = []
spend_feats = []
balance_feats = []
risk_feats = []

for feat in list(df_train):
    if feat in cat_features:
        continue
    
    if feat[0] == 'P':
        #print(feat)
        payment_feats.append(feat)
    elif feat[0] == 'D':
        delinq_feats.append(feat)
    elif feat[0] == 'S':
        spend_feats.append(feat)
    elif feat[0] == 'B':
        balance_feats.append(feat)
    elif feat[0] == 'R':
        risk_feats.append(feat)

In [ ]:
len(payment_feats) + len(delinq_feats) + len(spend_feats) + len(balance_feats) + len(risk_feats)

In [ ]:
num_features = payment_feats + delinq_feats + spend_feats + balance_feats + risk_feats
len(num_features)

### TabNet

In [1]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\oobur\\Projects\\spbu_master\\sem4\\input\\pytorchtabnet\\pytorch_tabnet-3.1.1-py3-none-any.whl'


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric

In [ ]:
clf = TabNetClassifier()  #TabNetRegressor()
clf.fit(
  X_train, Y_train,
  eval_set=[(X_valid, y_valid)]
)
preds = clf.predict(X_test)

In [ ]:
# пока попробуем онли на числовых, без fillna может заработает

In [ ]:
class CFG:
    DEBUG = False
    model = 'tabnet'
    N_folds = 5
    seed = 42
    batch_size = 512
    max_epochs = 60

In [ ]:
import os
import random
import psutil

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed = CFG.seed)

In [ ]:
psutil.virtual_memory().percent

In [ ]:
# using amex metric to evaluate tabnet
class Amex_tabnet(Metric):
    
  def __init__(self):
    self._name = 'amex_tabnet'
    self._maximize = True

  def __call__(self, y_true, y_pred):
    amex = get_amex_metric_calculated(y_true, y_pred[:, 1])
    return max(amex, 0.)

### Training

In [ ]:
print('\n ', '-'*50)
print('\nTraining: ', CFG.model)
print('\n ', '-'*50)


print('\nSeed: ', CFG.seed)
print('N folds: ', CFG.N_folds)

print('\nN features: ', len(df_train.columns.values.tolist()))
print('\n')

In [ ]:







# Create out of folds array
oof_predictions = np.zeros((train.shape[0]))
test_predictions = np.zeros(test.shape[0])
feature_importances = pd.DataFrame()
feature_importances["feature"] = train.columns.tolist()
stats = pd.DataFrame()
explain_matrices = []
masks_ =[]


    
kfold = StratifiedKFold(n_splits = CFG.N_folds, shuffle=True, random_state = CFG.seed)

for fold, (train_idx, valid_idx) in enumerate(kfold.split(train, target)):

    ## DEBUG MODE
    if CFG.DEBUG == True:
        if fold > 0:
            print('\nDEBUG mode activated: Will train only one fold...\n')
            break      

    start = time.time()

    X_train, y_train = train.loc[train_idx], target.loc[train_idx]
    X_valid, y_valid = train.loc[valid_idx], target.loc[valid_idx]        
        
    model = TabNetClassifier(n_d = 32,
                             n_a = 32,
                             n_steps = 3,
                             gamma = 1.3,
                             n_independent = 2,
                             n_shared = 2,
                             momentum = 0.02,
                             clip_value = None,
                             lambda_sparse = 1e-3,
                             optimizer_fn = torch.optim.Adam,
                             optimizer_params = dict(lr = 1e-3, weight_decay=1e-3),
                             scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
                             scheduler_params = {'T_0':5,
                                                 'eta_min':1e-4,
                                                 'T_mult':1,
                                                 'last_epoch':-1},
                             mask_type = 'entmax',
                             seed = CFG.seed)
    
    

    ## train
    model.fit(np.array(X_train),
              np.array(y_train.values.ravel()),
              eval_set = [(np.array(X_valid), np.array(y_valid.values.ravel()))],
              max_epochs = CFG.max_epochs,
              patience = 50,
              batch_size = CFG.batch_size,
              eval_metric = ['auc', 'accuracy', Amex_tabnet]) # Last metric is used for early stopping
    
    # Saving best model
    saving_path_name = f"./fold{fold}"
    saved_filepath = model.save_model(saving_path_name)
    
    # model explanability
    explain_matrix, masks = model.explain(X_valid.values)
    explain_matrices.append(explain_matrix)
    masks_.append(masks[0])
    masks_.append(masks[1])
    
    # Inference
    oof_predictions[valid_idx] = model.predict_proba(X_valid.values)[:, 1]
    
    #if CFG
    # logodds function
    
    test_predictions += model.predict_proba(test.values)[:, 1]/5
    feature_importances[f"importance_fold{fold}+1"] = model.feature_importances_
    
    # Loss , metric tracking
    stats[f'fold{fold+1}_train_loss'] = model.history['loss']
    stats[f'fold{fold+1}_val_metric'] = model.history['val_0_amex_tabnet']


    end = time.time()
    time_delta = np.round((end - start)/60, 2)
     
    print(f'\nFold {fold+1}/{CFG.N_folds} | {time_delta:.2f} min')

    ### free memory
    del X_train, y_train
    del X_valid, y_valid
    gc.collect()

print(f'OOF score across folds: {amex_metric_numpy(target, oof_predictions.flatten())}')